# Análise Exploratória de Dados (EDA) - Preços de Carros

**Objetivo:** Entender o conjunto de dados `carros.csv` para identificar padrões, correlações e preparar o terreno para a modelagem de um sistema de previsão de preços.

**Passos:**
1.  **Setup Inicial:** Importar bibliotecas e carregar os dados.
2.  **Inspeção Geral:** Verificar a estrutura, tipos de dados e estatísticas descritivas.
3.  **Análise Univariada:** Analisar cada variável individualmente.
4.  **Análise Bivariada:** Analisar a relação entre as variáveis, especialmente com a nossa variável alvo (`Preco`).
5.  **Conclusões e Próximos Passos:** Resumir os achados e definir a estratégia para o modelo.

## 1. Setup Inicial

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações de visualização
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# Carregando os dados
# O notebook está em /notebooks, então precisamos voltar um nível para acessar /data
df = pd.read_csv('../data/carros.csv')

## 2. Inspeção Geral

In [ ]:
# Visualizar as 5 primeiras linhas
df.head()

In [ ]:
# Informações sobre o DataFrame: tipos de dados e valores não nulos
df.info()

**Observação:** Não há valores nulos no nosso dataset. As colunas `Marca` e `Modelo` são do tipo `object` (texto), enquanto as outras são numéricas, como esperado.

In [ ]:
# Estatísticas descritivas para as colunas numéricas
df.describe()

**Observação:**
- O ano dos carros varia de 2014 a 2021.
- A quilometragem média é de cerca de 52.000 km.
- O preço médio é de R$ 46.750.

## 3. Análise Univariada

### Variáveis Categóricas (`Marca`, `Modelo`)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

sns.countplot(data=df, x='Marca', ax=ax[0], palette='viridis')
ax[0].set_title('Contagem de Carros por Marca')

sns.countplot(data=df, y='Modelo', ax=ax[1], palette='plasma')
ax[1].set_title('Contagem de Carros por Modelo')

plt.tight_layout()
plt.show()

**Observação:** O dataset está perfeitamente balanceado em termos de marcas e modelos, com 3 ocorrências para cada marca e 2 para cada modelo. Em um cenário real, isso raramente acontece.

### Variáveis Numéricas (`Ano`, `Quilometragem`, `Preco`)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 6))

sns.histplot(df['Ano'], kde=True, ax=ax[0], bins=8)
ax[0].set_title('Distribuição do Ano de Fabricação')

sns.histplot(df['Quilometragem'], kde=True, ax=ax[1])
ax[1].set_title('Distribuição da Quilometragem')

sns.histplot(df['Preco'], kde=True, ax=ax[2])
ax[2].set_title('Distribuição do Preço')

plt.show()

**Observação:** As distribuições nos dão uma ideia da faixa de valores para cada variável. A distribuição de preços parece ter dois "picos", o que pode indicar grupos diferentes de carros (talvez os mais novos vs. os mais antigos).

## 4. Análise Bivariada

### Relação das Features com o Preço

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

# Preço vs. Quilometragem
sns.scatterplot(data=df, x='Quilometragem', y='Preco', hue='Marca', s=100, ax=ax[0])
ax[0].set_title('Preço vs. Quilometragem')
ax[0].text(70000, 70000, 'Tendência Negativa Clara', fontsize=12, color='red')

# Preço vs. Ano
sns.scatterplot(data=df, x='Ano', y='Preco', hue='Marca', s=100, ax=ax[1])
ax[1].set_title('Preço vs. Ano')
ax[1].text(2014.5, 25000, 'Tendência Positiva Clara', fontsize=12, color='green')

plt.show()

**Hipóteses Fortes:**
1.  **Quilometragem:** Quanto maior a quilometragem, menor o preço. Existe uma correlação negativa forte e linear.
2.  **Ano:** Quanto mais novo o carro (maior o ano), maior o preço. Existe uma correlação positiva forte e linear.

Essas duas variáveis parecem ser excelentes preditoras para um modelo de regressão linear.

In [ ]:
# Analisando o preço por categoria
fig, ax = plt.subplots(1, 2, figsize=(18, 7))

sns.boxplot(data=df, x='Marca', y='Preco', ax=ax[0])
ax[0].set_title('Distribuição de Preço por Marca')

sns.boxplot(data=df, x='Modelo', y='Preco', ax=ax[1])
ax[1].set_title('Distribuição de Preço por Modelo')
plt.xticks(rotation=45)

plt.show()

**Observação:**
- **Marca:** Chevrolet e VW parecem ter carros com preços medianos mais altos que Fiat e Ford neste dataset.
- **Modelo:** Onix e Polo são os modelos mais caros, enquanto o Uno é o mais barato. 

Isso confirma que `Marca` e `Modelo` também são informações importantes que influenciam o preço e devem ser incluídas no nosso modelo.

### Matriz de Correlação

In [ ]:
# A matriz de correlação só funciona com dados numéricos
correlation_matrix = df.corr(numeric_only=True)

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlação entre Variáveis Numéricas')
plt.show()

**Confirmação:**
- `Preco` tem uma correlação positiva muito forte com `Ano` (**+0.96**).
- `Preco` tem uma correlação negativa muito forte com `Quilometragem` (**-0.95**).
- `Ano` e `Quilometragem` também são fortemente correlacionados entre si (**-0.93**), o que é esperado (carros mais velhos tendem a ter mais quilometragem).

## 5. Conclusões e Próximos Passos

1.  **Qualidade dos Dados:** O dataset é pequeno, mas limpo e sem valores ausentes.
2.  **Features Relevantes:** Todas as quatro features (`Marca`, `Modelo`, `Ano`, `Quilometragem`) mostram ter uma relação clara com a variável alvo (`Preco`).
3.  **Relações Lineares:** As relações entre as features numéricas (`Ano`, `Quilometragem`) e o `Preco` são visivelmente lineares. Isso sugere que um modelo de **Regressão Linear Múltipla** é um excelente ponto de partida.
4.  **Tratamento de Features:** As features categóricas (`Marca`, `Modelo`) precisam ser convertidas para um formato numérico para que o modelo de regressão possa utilizá-las. A técnica de **One-Hot Encoding** (como `pd.get_dummies`) é a abordagem padrão para isso.

**Estratégia de Modelagem:**
- **Features (X):** `Ano`, `Quilometragem`, e as versões One-Hot Encoded de `Marca` e `Modelo`.
- **Alvo (y):** `Preco`.
- **Modelo:** `LinearRegression` do Scikit-learn.